# Проект GoProtect

Цель проекта : Создать решение для стандартизации названий спортивных школ.\
Для достижения целей намечены следующиен задачи:
+ Загрузка данных
+ Применение предобученной модели на данных
+ Формирование тренировочного датасета
+ Дообучение модели
+ Анализ качества модели после дообучения.

Проект выполнил - Скобников В.А (https://t.me/vladskoba)


In [35]:
!pip install --upgrade jupyter -q

In [36]:
!pip install --upgrade ipywidgets -q

In [37]:
!pip install transformers -q
!pip install torch -q
!pip install scikit-learn -q
!pip install pandas -q

In [38]:
!pip install transformers[torch] -q
!pip install accelerate -U -q

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModel,BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics.pairwise import cosine_similarity
from itertools import product
from joblib import Parallel, delayed
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset

## Загрузка данных

Загрузим данные и посмотрим, что ои из себя представляют.

In [40]:
some_names_df = pd.read_csv("D:\DS\data_02_match_school\data_02_match_school\Примерное написание.csv")  
true_names_df = pd.read_csv("D:\DS\data_02_match_school\data_02_match_school\Школы.csv")  

In [41]:
some_names_df.head()

,school_id,name
0,1836,"ООО ""Триумф"""
1,1836,"Москва, СК ""Триумф"""
2,610,"СШОР ""Надежда Губернии"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,609,"""СШ ""Гвоздика"""


In [42]:
true_names_df.head()

,school_id,name,region
0,1,Авангард,Московская область
1,2,Авангард,Ямало-Ненецкий АО
2,3,Авиатор,Республика Татарстан
3,4,Аврора,Санкт-Петербург
4,5,Ice Dream / Айс Дрим,Санкт-Петербург


In [68]:
some_names_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school_id   895 non-null    int64 
 1   name        895 non-null    object
 2   best_match  895 non-null    object
dtypes: int64(1), object(2)
memory usage: 21.1+ KB


In [69]:
true_names_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  306 non-null    int64 
 1   name       306 non-null    object
 2   region     306 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.3+ KB


Есть некоторый набор, содержащий в себе множество различный наименований для конкретного заведения. И есть список с эталонными значениями. 

## Загрузка модели и токенайзера от Hugging Face

Посмотрим, как модель себя проявит без дообучения :

### Загрузка модели

In [43]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

### Получения эмбеддингов

In [44]:
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings

In [45]:
some_names_embeddings_1 = get_embeddings(some_names_df['name'].tolist())
true_names_embeddings_1 = get_embeddings(true_names_df['name'].tolist())

### Оценка косинусного расстояния

In [46]:
similarity_matrix = cosine_similarity(some_names_embeddings_1, true_names_embeddings_1)

### Установление максимально близких связей

In [71]:
matches = similarity_matrix.argmax(axis=1)
some_names_df['best_match'] = [true_names_df['name'].iloc[i] for i in matches]

some_names_df[['name', 'best_match']]

,name,best_match
0,"ООО ""Триумф""","ООО ""Триумф"""
1,"Москва, СК ""Триумф""","ООО ""Триумф"""
2,"СШОР ""Надежда Губернии","СШОР ""Надежда Губернии"
3,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...","СШОР ""Надежда Губернии"
4,"""СШ ""Гвоздика""","""СШ ""Гвоздика"""
...,...,...
890,"Республика Татарстан, СШОР ФСО Авиатор",Центр развития спорта
891,"СШОР ФСО Авиатор, Республика Татарстан",Фаворит
892,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»",Фаворит
893,"ЯНАО, СШ ""Авангард""",Авангард


Можно заметить, что такой подход позволяет установить связи между "вариативным" и "эталонным". Другой вопрос, что адекватно оценить такую модель метриками сложно, ввиду отсутвия меток. Для более серьёзного решения создадим тренировочный датасет.

## Создание  тренировочного датасета

Далее создадим тренировочный датасет с помощью метода TF-IDF  для предварительного отбора наиболее вероятных соответствий.

### Использование TF-IDF для предварительного отбора

In [48]:
tfidf_vectorizer = TfidfVectorizer().fit(pd.concat([some_names_df['name'], true_names_df['name']]))
some_names_tfidf = tfidf_vectorizer.transform(some_names_df['name'])
true_names_tfidf = tfidf_vectorizer.transform(true_names_df['name'])

In [49]:
cosine_similarities = linear_kernel(some_names_tfidf, true_names_tfidf)

In [50]:
top_n = 5
top_n_indices = np.argsort(-cosine_similarities, axis=1)[:, :top_n]

candidate_pairs = []
for inst_idx, ref_indices in enumerate(top_n_indices):
    inst_id = some_names_df.iloc[inst_idx]['school_id']
    inst_name = some_names_df.iloc[inst_idx]['name']
    for ref_idx in ref_indices:
        ref_id = true_names_df.iloc[ref_idx]['school_id']
        ref_name = true_names_df.iloc[ref_idx]['name']
        candidate_pairs.append({
            'some_names_id': inst_id,
            'true_names_id': ref_id,
            'name_some': inst_name,
            'name_true': ref_name
        })

candidate_pairs_df = pd.DataFrame(candidate_pairs)

### Создание положительных и отрицательных примеров из кандидатов

In [51]:
# Создание меток (1 если идентификаторы совпадают, 0 в противном случае)
candidate_pairs_df['label'] = (candidate_pairs_df['some_names_id'] == candidate_pairs_df['true_names_id']).astype(int)

In [52]:
# Выборка отрицательных примеров
positive_pairs = candidate_pairs_df[candidate_pairs_df['label'] == 1]
negative_pairs = candidate_pairs_df[candidate_pairs_df['label'] == 0].sample(n=len(positive_pairs))

In [53]:
# Объединение положительных и отрицательных примеров
train_dataset = pd.concat([positive_pairs, negative_pairs])

### Создание эмбеддингов и вычисление косинусного сходства

In [54]:
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings

In [55]:
some_embeddings = get_embeddings(train_dataset['name_some'].tolist())
true_embeddings = get_embeddings(train_dataset['name_true'].tolist())

In [56]:
# Вычисление косинусного сходства
similarities = cosine_similarity(some_embeddings, true_embeddings)
train_dataset['similarity'] = similarities.diagonal()

In [57]:
train_dataset.head()

,some_names_id,true_names_id,name_some,name_true,label,similarity
0,1836,1836,"ООО ""Триумф""","ООО ""Триумф""",1,0.998387
6,1836,1836,"Москва, СК ""Триумф""","ООО ""Триумф""",1,0.951282
10,610,610,"СШОР ""Надежда Губернии","СШОР ""Надежда Губернии",1,0.997189
15,610,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе...","СШОР ""Надежда Губернии",1,0.848917
20,609,609,"""СШ ""Гвоздика""","""СШ ""Гвоздика""",1,0.998699


Этот подход позволяют сократить пространство поиска и уменьшить количество пар для обработки, что значительно снижает нагрузку на память и вычислительные ресурсы, чем если бы мы пребирали все возможные соответствия.

## Дообучение модели

Для этого мы будем использовать метод обучения, известный как fine-tuning, на задаче бинарной классификации. Мы будем использовать модель sentence-transformers и библиотеку transformers для этой цели.

### Подготовка датасета и разделение на выборки

In [58]:
train_df, val_df = train_test_split(train_dataset, test_size=0.1, random_state=42)


In [59]:
def prepare_data(df):
    return list(zip(df['name_some'].tolist(), df['name_true'].tolist(), df['label'].tolist()))

train_data = prepare_data(train_df)
val_data = prepare_data(val_df)

### Создание модели и токенайзера

Не будем подгружать в модель столбцы с id школ. Это ухудшит метрику, но сделает исследование честнее.

In [80]:
class InstitutionDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.labels = df['label'].values
        self.texts = df[['name_some', 'name_true']].values
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text1, text2 = self.texts[idx]
        inputs = self.tokenizer(text1, text2, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        inputs = {key: val.squeeze(0) for key, val in inputs.items()}
        inputs['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return inputs


In [81]:
model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/paraphrase-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [82]:
train_dataset = PairDataset(train_data, tokenizer)
val_dataset = PairDataset(val_data, tokenizer)

### Определение функции вычисления метрик

In [83]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
    acc = accuracy_score(p.label_ids, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

### Настройка тренировочного процесса

In [84]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps"
)

D:\Anaconda\envs\envel\lib\site-packages\transformers\training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Обучение модели

In [85]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
10,0.694400,0.690441,0.538462,0.538462,1.000000,0.700000
20,0.696500,0.690426,0.538462,0.538462,1.000000,0.700000
30,0.690300,0.690240,0.538462,0.538462,1.000000,0.700000
40,0.693300,0.689989,0.538462,0.538462,1.000000,0.700000
50,0.695200,0.689799,0.538462,0.538462,1.000000,0.700000
60,0.691600,0.689618,0.538462,0.538462,1.000000,0.700000
70,0.692500,0.689602,0.628205,0.614035,0.833333,0.707071
80,0.689800,0.688993,0.538462,0.615385,0.380952,0.470588
90,0.689400,0.685582,0.621795,0.605042,0.857143,0.709360
100,0.684300,0.681813,0.634615,0.628571,0.785714,0.698413


TrainOutput(global_step=264, training_loss=0.6263205919301871, metrics={'train_runtime': 213.6517, 'train_samples_per_second': 19.714, 'train_steps_per_second': 1.236, 'total_flos': 34923597576192.0, 'train_loss': 0.6263205919301871, 'epoch': 3.0})

### Оценка модели

In [86]:
metrics = trainer.evaluate()
metrics

{'eval_loss': 0.5766525268554688,
 'eval_accuracy': 0.7115384615384616,
 'eval_precision': 0.8421052631578947,
 'eval_recall': 0.5714285714285714,
 'eval_f1': 0.6808510638297872,
 'eval_runtime': 2.2885,
 'eval_samples_per_second': 68.168,
 'eval_steps_per_second': 1.311,
 'epoch': 3.0}

## Общий вывод

В ходе работы  было использовано два подхода : использование модели 'all-MiniLM-L6-v2' "из коробки" и с дообучением. Оба варианта так или иначе выполняют свою работу, однако только второй вариант можно как-то оценить - **(Метрики Accuracy:0.71,Precision:0.84 ,Recall:0.57, F1:0.68)**.
Качество модели очевидно можно улучшить путём увеличения данных для обучения. В перспективе работы есть создание демо приложения по существующей модели.